In [1]:
import pandas as pd
import geopandas as gpd

import shared_utils
from utils import *
import intake
import gcsfs

import calitp
from calitp.tables import tbl
from siuba import *

/Users/edasmalchi/miniconda3/envs/calitp/lib/python3.10/site-packages/google/cloud/bigquery/client.py:535: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [2]:
coverage_gdf = gpd.read_parquet(f'{GCS_FILE_PATH}block_accessibility_metrics.parquet')

In [3]:
coverage_gdf.head(3)

,county,tract,block,geo_id,geometry,block_pop,area,rt_accessible,rt,accessible,static
0,17,30706,3012,60170307063012,"POLYGON ((-93795.217 73341.696, -93626.457 734...",363,83581.711289,False,False,False,False
1,17,30601,3006,60170306013006,"POLYGON ((-88050.178 97425.705, -88018.654 977...",64,933640.239287,False,False,False,False
2,65,42628,4008,60650426284008,"POLYGON ((257831.965 -463783.472, 258008.040 -...",103,15986.958897,False,False,False,False


In [12]:
# popup_dict = {
#     "rt_accessible": "Accessible Trip and Stop, RT Feed Available",
#     "rt": "RT Feed Available",
#     "accessible": "Accessible Trip and Stop",
#     "static": "GTFS Static Present",
# }

popup_dict = {}

In [13]:
import branca

In [14]:
from shared_utils import calitp_color_palette as cp

In [15]:
cp.CALITP_CATEGORY_BRIGHT_COLORS

['#2EA8CE', '#EB9F3C', '#F4D837', '#51BF9D', '#8CBCCB', '#9487C0']

In [17]:
coverage_gdf.geometry = coverage_gdf.buffer(0)

In [18]:
df_rt_acc = (coverage_gdf.copy() >> filter(_.rt_accessible)).dissolve()
df_rt = (coverage_gdf.copy() >> filter(_.rt)).dissolve()
df_acc = (coverage_gdf.copy() >> filter(_.accessible)).dissolve()
df_static = (coverage_gdf.copy() >> filter(_.static)).dissolve()

In [19]:
LAYERS_DICT = {
    "RT Accessible": {"df": df_rt_acc,
        "plot_col": "rt_accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[:1],
    },
    "RT": {"df": df_rt,
        "plot_col": "rt",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[1:2],
    },
    "Accessible": {"df": df_acc,
        "plot_col": "accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[2:3],
    },
    "GTFS Static": {"df": df_static,
        "plot_col": "static",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[3:4],
    }
}

In [20]:
# shared_utils.map_utils.make_folium_multiple_layers_map?

In [21]:
fig = shared_utils.map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 800, fig_height = 800, 
        zoom=9, 
        centroid = [36.5, -120], 
        title=f"Statewide Feed Coverage",
        # legend_dict = LEGEND_DICT
    )

In [ ]:
# fig

In [22]:
fig.save('statewide_coverage_map.html')